/**********************************************************************************
 * Copyright (c) 2023 Process Systems Engineering (AVT.SVT), RWTH Aachen University
 *
 * This program and the accompanying materials are made available under the
 * terms of the Eclipse Public License 2.0 which is available at
 * http://www.eclipse.org/legal/epl-2.0.
 *
 * SPDX-License-Identifier: EPL-2.0
 *
 * The source code can be found here:
 * https://github.com/iAMB-RWTH-Aachen/Ustilago_maydis-GEM/tree/master/data/AcetateCofeed
 **********************************************************************************/


Date: 04.02.2023 <br/>
File name: iUma22_testing_knockouts.ipynb <br/>
Description: This script provides the FBA computations with implemented knockouts that were suggested by OptKnock (knockouts displayed in Figure 5 in the original manuscript).
The COBRApy package (https://doi.org/10.1186/1752-0509-7-74  ) is used to together with the solver Gurobi 9.5.2 to solve the 
linear programming problems. <br/>
@authors: Manuel Boßmann, Anita Ziegler, Ulf Liebal, Alexander Mitsos


In [1]:
from os.path import join
from cobra.io import read_sbml_model

#load iUma22v1.1.xml model
ModelFile = join('..', '..','model','iUma22v1.1.xml')
model=read_sbml_model(ModelFile)

In [2]:
model.objective = "BIOMASS_REACTION"

#3 knockouts as predicted using OptKnock. See paper, section:
#"Improving the itaconic acid yield by knocking out three reactions"
model.reactions.get_by_id('HISTD').knock_out()
model.reactions.get_by_id('TRANS_Adenine_Nucleotide_Transporter').knock_out()
model.reactions.get_by_id('MDH').knock_out()

#This is a two step optimization: First, biomass growth rate is the optimization objective. Then, the biomass growth rate is set as a lower bound.
#Then itaconic acid is set as the optimization objective.
solution = model.optimize()

biomass_flux = solution.fluxes.BIOMASS_REACTION

In [3]:
#Biomass growth rate in 1/h:
solution.fluxes.BIOMASS_REACTION

0.02887541869357128

In [4]:
#Keep grwoth rate from simulation before, where biomass was the objective
model.reactions.BIOMASS_REACTION.lower_bound = biomass_flux

#Set itaconic acid as new objective
model.objective = "EX_itacon_e"

In [5]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,2.2,6,100.00%
nh4_e,EX_nh4_e,0.136,0,0.00%
o2_e,EX_o2_e,0.9734,0,0.00%
so4_e,EX_so4_e,0.002368,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
12ppd__R_c,DM_12ppd__R_c,-1.896,3,35.44%
actn__R_c,DM_actn__R_c,-0.1555,4,3.88%
adn_e,EX_adn_e,-0.4083,10,25.45%
co2_e,EX_co2_e,-3.013,1,18.78%
h2o_e,EX_h2o_e,-1.238,0,0.00%


In [6]:
#The resulting itaconic acid flux:
model.reactions.EX_itacon_e.flux

0.5282015957982129